Numerical Methods Homework 2, due 10/6/22

Kerry Hall

In [1]:
#Unless otherwise noted, code present was constructed by and is the explicit property of Kerry Hall. All conclusions present are property of Kerry Hall. 
import numpy as np
from numpy import array, zeros, fabs
import time
from scipy import linalg

In [2]:
# Below function provided by Dr. Zhang and edited by Kerry Hall
def gauss_elimination(square_mat, col):
    n = len(col)
    x = zeros(n, float)
    #first loop specifys the fixed row
    for k in range(n-1):
        if fabs(square_mat[k,k]) < 1.0e-12:
            for i in range(k+1, n):
                if fabs(square_mat[i,k]) > fabs(square_mat[k,k]):
                    square_mat[[k,i]] = square_mat[[i,k]]
                    col[[k,i]] = col[[i,k]]
                    break
     #applies the elimination below the fixed row
        for i in range(k+1,n):
            if square_mat[i,k] == 0:continue
            factor = square_mat[k,k]/square_mat[i,k]
            for j in range(k,n):
                square_mat[i,j] = square_mat[k,j] - square_mat[i,j]*factor
                #we also calculate the b vector of each row
            col[i] = col[k] - col[i]*factor
    # substitution backward
    x[n-1] = col[n-1] / square_mat[n-1, n-1]
    for i in range(n-2, -1, -1):
        sum_ax = 0
        for j in range(i+1, n):
            sum_ax += square_mat[i,j] * x[j]
        x[i] = (col[i] - sum_ax) / square_mat[i,i]
    return x

In [3]:
# Below function provided by Dr. Zhang and edited by Kerry Hall to be recursive
def seidel(square_mat, guess_col, col, iter_=25, count=0):
    n = len(square_mat)
    # for loop for 3 times as to calculate x, y , z
    for j in range(0, n):
        # temp variable d to store b[j]
        d = col[j]

        # to calculate respective xi, yi, zi
        for i in range(0, n):
            if(j != i): d-= square_mat[j][i] * guess_col[i]
        # updating the value of our solution
        guess_col[j] = d / square_mat[j][j]
    # returning our updated solution
    if count == iter_: return guess_col
    else: return seidel(square_mat, guess_col, col, count=count+1)

In [4]:
# Below function provided by Dr. Zhang and edited by Kerry Hall to become recursive
def jacobian(A, b, iter_=25, count=0, x=0):
    if count == 0: x=np.zeros_like(b)
    for it_count in range(iter_):
        x_new = np.zeros_like(x)
        for i in range(A.shape[0]):
            s1 = np.dot(A[i, :i], x[:i])
            s2 = np.dot(A[i, i + 1:], x[i + 1:])
            x_new[i] = (b[i] - s1 - s2) / A[i, i]
    count += 1
    if np.allclose(x, x_new, atol=1e-10, rtol=0.0): return x_new
    elif count == iter_: return x_new
    else: return jacobian(A, b, x=x_new, iter_=iter_, count=count)

In [5]:
A1 = np.array([[2., -2., -1.,],
              [4.,  1., -2.,],
              [-2., 1., -1.,],])
B1 = np.array([-2., 1., -3.])

#The seidel guess below is extremely close to the right answer, yet the seidel function still returns a diverging value due to non-diagonality of the a matrix
s1 = seidel(A1, np.array([1., 1.000000000000001, 2.]) , B1)
ge1 = gauss_elimination(A1, B1)
np_sol1 = np.linalg.solve(A1, B1)
print("\nThe Gaussian Elimination yields the vector: ", ge1,
      "\n\nThe Seidel method yields the vector: ", s1,
      "\n\nThe NumPy linalg module yields the vector: ", np_sol1, "\n", sep="")


The Gaussian Elimination yields the vector: [1. 1. 2.]

The Seidel method yields the vector: [-1.5  6.  12. ]

The NumPy linalg module yields the vector: [1. 1. 2.]



In [6]:
A2 = np.array([[1., 5., 2.,],
               [3., 1., 6.,],
               [8., 3., 4.,],])
B2 = np.array([0., 3., 1.,])
LU = linalg.lu(A2)

#Seidel function requires diagonal matrix to be fed
tic = time.perf_counter()
s2 = seidel(LU[1] @ LU[2], np.array([.1,-.2,.6]), LU[0] @ (LU[0] @ B2)) #rotating twice 
toc = time.perf_counter()
siedel_time = toc - tic

tic = time.perf_counter()
j2 = jacobian(LU[1] @ LU[2], LU[0] @ (LU[0] @ B2), iter_=55) #55 iterations required to reach the same result as seidel and gaussian methods
toc = time.perf_counter()
jacobian_time = toc - tic

tic = time.perf_counter()
ge2 = gauss_elimination(A2, B2)
toc = time.perf_counter()
gauss_time = toc - tic

tic = time.perf_counter()
np_sol2 = np.linalg.solve(A2, B2)
toc = time.perf_counter()
np_time = toc - tic

print("\nThe Gaussian Elimination solution is: ", ge2, 
      "\n\nThe lower triangular matrix is:\n", LU[1], 
      "\n\nThe upper triangular matrix is:\n", LU[2],
      "\n\nThe Jacobian method solution is: ", j2,
      "\n\nThe Seidel method solution is: ", s2,
      "\n\nThe NumPy linalg solution is: ", np_sol2, sep="")
print("\nThe runtime in milliseconds for the Gaussian, Jacobian, Seidel, and NumPy methods respectively is: ", 
      1000*gauss_time, 1000*jacobian_time, 1000*siedel_time, 1000*np_time, sep="\n\n")


The Gaussian Elimination solution is: [-0.08333333 -0.21428571  0.57738095]

The lower triangular matrix is:
[[ 1.          0.          0.        ]
 [ 0.125       1.          0.        ]
 [ 0.375      -0.02702703  1.        ]]

The upper triangular matrix is:
[[8.         3.         4.        ]
 [0.         4.625      1.5       ]
 [0.         0.         4.54054054]]

The Jacobian method solution is: [-0.08333333 -0.21428571  0.57738095]

The Seidel method solution is: [-0.08333333 -0.21428571  0.57738095]

The NumPy linalg solution is: [-0.08333333 -0.21428571  0.57738095]

The runtime in milliseconds for the Gaussian, Jacobian, Seidel, and NumPy methods respectively is: 

0.11470000026747584

108.33099999581464

0.9452000085730106

0.11799999629147351


Based on the results above, it is very apparent that the Seidel/Jacobian methods fail on non-diagonal matrices. This is due to the convergence assumptions made. The Jacobian method is also around 2 orders of magnitude larger than the Seidel method in terms of the computational time cost. The Jacobian method also requires many more iterations than the other methods. The Seidel method converges at an acceptable rate, but it is still almost an order of magnitude slower than Gaussian elimination. This leads me to believe that the best solution is to simply use Gaussian elimination. The Siedel and Jacobian matrices also have the added time cost of computing the permutated matrix for computation. It is very interesting to me that the NumPy solver is basically the same speed as the Gaussian elimination. This indicates to me that NumPy may be calling very similar code to the defined function for Gaussian Elimination. 